In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyksc import ksc

In [ ]:
def images_path():
    path = os.getcwd()
    path = path + '/../../images'
    
    return(path)

In [ ]:
FILENAME='/home/wagnernegrao/Downloads/dataset.csv'

In [ ]:
# load dataset
df = pd.read_csv(FILENAME, usecols=['Date', 'Project', 
                                    'code', 'filename', 'language'])



In [ ]:
# removes dirt from the Date column 
df['Date'] = df.Date.apply(lambda date: date.replace('.csv', ''))

In [ ]:
# convert column to datetime
df['Date'] = pd.to_datetime(df['Date'])

# sort column
df = df.sort_values(by='Date')

In [ ]:
# filter only the languages that are being used
df = df.loc[(df['language']  == 'C') |
              (df['language'] == 'C++') |
              (df['language'] == 'C#') |
              (df['language'] == 'Clojure') |
              (df['language'] == 'CoffeeScript') |
              (df['language'] == 'Go') |
              (df['language'] == 'Haskell') |
              (df['language'] == 'Java') |
              (df['language'] == 'JavaScript') |
              (df['language'] == 'Kotlin') |
              (df['language'] == 'Lua') |
              (df['language'] == 'Objective C') |
              (df['language'] == 'PHP') |
              (df['language'] == 'Python') |
              (df['language'] == 'Ruby') |
              (df['language'] == 'Rust') |
              (df['language'] == 'Scala') |
              (df['language'] == 'Swift') |
              (df['language'] == 'TypeScript') |
              (df['language'] == 'Elixir')]

In [ ]:
def file_unique(dfx):
    """
    Cria uma lista com arquivos unicos e remove os arquivos repetidos
    
    Parameters:
    ----------
    object dfx: recebe um dataset
    
    Return:
    -------
    list files: lista com os arquivos unicos
    """
    
    files_list = dfx.filename.tolist()

    files = []

    for i in files_list:
        if(i not in files):
            files.append(i)
    
    return(files)

In [ ]:
# Filtra o dataframe para apenas arquivos acima do limiar
dfx = df.loc[df['code'] >= 13196]


unique_files = file_unique(dfx)

In [ ]:
print("Quantidade de arquivos unicos: {}".format(len(unique_files)))

In [ ]:
def create_dataframe(df, unique_files):
    '''
    Cria um dataframe para cada arquivo unico e adiciona numa lista. 
    
    Parameters:
    -----------
    object df: Dataset
    list unique_files: Lista com os arquivos unicos
    
    Return:
    -------
    list dfs: Uma lista com varios dataframes
    '''
    
    dfs = []
    
    for path in unique_files:
         dfs.append(df.loc[df['filename'] == path])
    
    return(dfs)

In [ ]:
# Torna float a coluna code
dfx['code'] = dfx.code.apply(lambda code: float(code))

In [ ]:
dfs = create_dataframe(dfx, unique_files)

In [ ]:
def create_series(dfs):
    '''
    Percorre cada dataframe da lista e adiciona o LOC
    e os anos de cada um em uma lista.
    
    Parameters:
    -----------
    object dfs: Lista com dataframes
    
    Return:
    -------
    list tm_series: Lista com o LOC de cada arquivo
    list years: Lista com os anos de cada arquivo
    '''
    
    time_series = [] 
    years = [] 
    
    for df in dfs:
        time_series.append(df.code.tolist())
        years.append(df.Date.dt.year.tolist())
        
    return(time_series, years)


loc_list, years = create_series(dfs)

In [ ]:
import statistics as stacs

# Cria a media e a mediana

value_sum = 0
value_list = []

for i in loc_list:
    value_sum+=len(i)
    value_list.append(len(i))
    
value_list = sorted(value_list)

min_value = value_sum/len(loc_list)

print("Media de LOC: {}".format(value_sum/len(loc_list)))
print("Mediana de LOC: {}".format(stacs.median(value_list)))

In [ ]:
# Usado para deixar as listas com o mesmo tamanho 

def size_series(loc_list, min_value):
    
    """
    Verifica as listas que possuem o tamanho menor que o valor minimo
    e remove, apos isso deixa todas as listas com o mesmo tamanho
    
    Parameters:
    -----------
    list loc_list: Lista com todos os LOC dos arquivos
    int min_value: Valor minimo para tamanho das listas
    
    Return:
    -------
    list loc_list: Nova lista de LOC
    int min_value: Novo valor minimo
    
    """
    
    flag = False

    while(flag != True):
        
        for i in range(len(loc_list)):
            
            if(len(loc_list[i]) < min_value):
                loc_list.pop(i)
                break

            for j in range(len(loc_list[i])):
                if(j >= min_value):
                    loc_list[i].pop(j)
                    
                    break
                    
        for serie in loc_list:
            if(len(serie) == min_value):
                flag = True
            else:
                flag = False
                break
        
    # para todas séries iniciarem em zero, como sendo a criacao do arquivo
    for serie in loc_list:
        serie.insert(0,0)
        
    return(loc_list, min_value+1)

In [ ]:
new_loc_list, new_min_value = size_series(loc_list, min_value)

In [ ]:
def create_year_serie(min_value, years):
    """
    Cria uma lista com a quantidade de anos, apos isso adiciona
    novamente cada ano ate a lista ter o tamanho necessario
    
    Parameters:
    -----------
    int min_value: Valor minimo para tamanho da lista
    list years: Lista de anos de cada arquivo
    
    Return:
    -------
    list new_year: Lista de anos conforme o min value
    
    """
    
    new_year = [] 

    for year in years:
        for i in year:
            if i not in new_year:
                new_year.append(i)

    cont = 0 
    while(len(new_year) < min_value):
        new_year.append(new_year[cont])
        cont+=1

    new_year = sorted(new_year)
    
    return(new_year)

### Curva de Crescimento

In [ ]:
# para gerar esses plots deve ser em float o loc
tm_series = np.array(new_loc_list) #Cria uma matriz com os valores de LOC

clusters_by_time_series = {}

centroids, assign, best_shift, cent_dists = ksc.ksc(tm_series, 3) # inicia o algoritmo com 3 clusters


In [ ]:
if assign is not None:
    for series, cluster in zip(tm_series, assign):
        if cluster in clusters_by_time_series.keys():
            clusters_by_time_series[cluster].append(series)
        else:
            clusters_by_time_series[cluster] = [series]

In [ ]:
def plot_cluster(clusters_by_time_series, years):
    for cluster in clusters_by_time_series.keys():
        figure = plt.figure()

        for project_time_series in clusters_by_time_series[cluster]:

            project_time_series = [0 if i == 0.1 else int(i) for i in project_time_series]

            _years = create_year_serie(len(project_time_series), years)

            plt.plot(_years, project_time_series, color='black')

        plt.xlabel('Years')
        plt.ylabel('Line Of Code')

        path = images_path()
        filename = path + "/cluster/cluster_" + str(cluster) + ".eps"

        figure.savefig(filename, bbox_inches='tight', format='eps', dpi=1000)

In [ ]:
def plot_centroid(centroids, years):
    for cluster, centroid in zip(range(0, 3), centroids):
        growth_rate = centroid[0] + centroid[-1] * 100
        
        _years = create_year_serie(len(centroid), years)
        
        figure = plt.figure()
        plt.plot(_years, centroid, color='black')
        plt.xlabel('Years')
        plt.ylabel('Average')

        path = images_path()
        filename = path + "/centroids/centroids_" + str(cluster) + ".eps"

        figure.savefig(filename, bbox_inches='tight', format='eps', dpi=1000)

In [ ]:
plot_cluster(clusters_by_time_series, years)

In [ ]:
plot_centroid(centroids, years)

In [ ]:
# quantidade de arquivos por cluster

print("Quantidade de Arquivos {}".format(len(new_loc_list)))
print("\n")
for i in range(len(clusters_by_time_series)):
    print("Cluster {} possui {} arquivos, representa {}%".format(i+1, len(clusters_by_time_series[i]), round((float(len(clusters_by_time_series[i])) * 100)/float(len(new_loc_list)), 2)))

In [ ]:
'''
Para o o google sheets entenda a entrada dos dados 
a lista de centroids deve estar ordenada.
'''

'''
centroids_order = [[],[]]

for i in range(len(centroids)):
    for j in range(len(centroids[i])):
        centroids_order[i].append(round(centroids[i][j], 3))

#centroids_order
'''